In [2]:
#import 
import pandas as pd
import numpy as np

In [1]:
from sentence_transformers import SentenceTransformer
import sentencepiece as spm
import tqdm as notebook_tqdm

sentences = [
    "PKSHA Technologyは機械学習/深層学習技術に関わるアルゴリズムソリューションを展開している。",
    "この深層学習モデルはPKSHA Technologyによって学習され、公開された。",
    "広目天は、仏教における四天王の一尊であり、サンスクリット語の「種々の眼をした者」を名前の由来とする。",
]

model = SentenceTransformer('pkshatech/GLuCoSE-base-ja')
embeddings = model.encode(sentences)
print(embeddings)

c:\Users\user\JLab\EmojiSearch\EmojiSearch_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\JLab\EmojiSearch\EmojiSearch_venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\JLab\EmojiSearch\EmojiSearch_venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[-0.6769513   0.01856877  0.05583429 ... -0.16493748  0.2736825
  -0.08397228]
 [-0.6831903   0.22736116  0.15810858 ...  0.03275613 -0.02415418
   0.2763753 ]
 [ 0.01181644 -0.21357438  0.11399277 ... -0.4597569   0.33000684
  -0.51123774]]


In [3]:
#full_emoji.csv を読み込む
full_emoji = pd.read_csv("Data/full_emoji.csv")
full_emoji.head()
emojis_data = full_emoji.drop(columns=["Apple", "Google", "Facebook","Windows","Twitter","JoyPixels","Samsung","Gmail","SoftBank","DoCoMo","KDDI"], axis=1)


In [4]:
from googletrans import Translator
translator = Translator()
trans_en = translator.translate(text='はじめまして。私の名前は太郎です。', dest="en")
print(trans_en.text)

nice to meet you. My name is Taro.


In [5]:
import pandas as pd
from googletrans import Translator, LANGUAGES



# Translatorオブジェクトを作成
translator = Translator()

# 翻訳関数の定義
def translate_to_japanese(text):
    try:
        translated_text = translator.translate(text, src='en', dest='ja')
        return translated_text.text
    except Exception as e:
        print(f"翻訳中にエラーが発生しました: {e}")
        return text

trans_en = translator.translate(text='dog', dest='ja')
print(trans_en)

# 'name'列の各値を日本語に翻訳して新しい列に追加
emojis_data['name_ja'] = emojis_data['name'].apply(translate_to_japanese)

print(emojis_data)


Translated(src=en, dest=ja, text=犬, pronunciation=Inu, extra_data="{'translat...")
         #    emoji                                            unicode  \
0        1        😀                                            U+1F600   
1        2        😃                                            U+1F603   
2        3        😄                                            U+1F604   
3        4        😁                                            U+1F601   
4        5        😆                                            U+1F606   
...    ...      ...                                                ...   
1811  1812       🇿🇲                                    U+1F1FF U+1F1F2   
1812  1813       🇿🇼                                    U+1F1FF U+1F1FC   
1813  1814  🏴󠁧󠁢󠁥󠁮󠁧󠁿  U+1F3F4 U+E0067 U+E0062 U+E0065 U+E006E U+E006...   
1814  1815  🏴󠁧󠁢󠁳󠁣󠁴󠁿  U+1F3F4 U+E0067 U+E0062 U+E0073 U+E0063 U+E007...   
1815  1816  🏴󠁧󠁢󠁷󠁬󠁳󠁿  U+1F3F4 U+E0067 U+E0062 U+E0077 U+E006C U+E007...   

                            

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('pkshatech/GLuCoSE-base-ja')


emoji_embeddings = model.encode([ed for ed in emojis_data["name_ja"]])




c:\Users\user\JLab\EmojiSearch\EmojiSearch_venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [82]:
input_word = "猫　悲しい"

# 入力単語の埋め込みを生成
input_embedding = model.encode([input_word])

# 絵文字の埋め込みと入力単語の埋め込みを正規化
emoji_embeddings_norm = emoji_embeddings / np.linalg.norm(emoji_embeddings, axis=1, keepdims=True)
input_embedding_norm = input_embedding / np.linalg.norm(input_embedding)

# コサイン類似度を計算
cosine_similarities = np.dot(emoji_embeddings_norm, input_embedding_norm.T)

# コサイン類似度が高いものから順にインデックスをソート
closest_emoji_indices = np.argsort(-cosine_similarities, axis=0)[:5].flatten()

# 上位5つの絵文字を取得
closest_emojis = emojis_data.iloc[closest_emoji_indices]

# 結果を出力
print(f"入力単語「{input_word}」に最も近い絵文字:")
for index, row in closest_emojis.iterrows():
    emoji_row = emojis_data.iloc[index]
    print(f"{row['emoji']} :{emoji_row['name']}: (類似度: {cosine_similarities[index][0]:.3f})")


入力単語「猫　悲しい」に最も近い絵文字:
😿 :crying cat: (類似度: 0.843)
🙀 :weary cat: (類似度: 0.802)
😼 :cat with wry smile: (類似度: 0.779)
😹 :cat with tears of joy: (類似度: 0.732)
🐈 :cat: (類似度: 0.729)


In [27]:

input_word = "ひよこ"

input_embedding = model.encode([input_word])


In [28]:
# 最も近い絵文字を見つける
distances = np.linalg.norm(emoji_embeddings - input_embedding, axis=1)
closest_emoji_index = np.argmin(distances)
closest_emoji = emojis_data.iloc[closest_emoji_index]["emoji"] 


print(f"入力単語「{input_word}」に最も近い絵文字: {closest_emoji}")

入力単語「ひよこ」に最も近い絵文字: 🐤


In [35]:
#full_emoji.csv を読み込む
full_emoji = pd.read_csv("Data/full_emoji.csv")
full_emoji.head()
emojis_data = full_emoji.drop(columns=["Apple", "Google", "Facebook","Windows","Twitter","JoyPixels","Samsung","Gmail","SoftBank","DoCoMo","KDDI"], axis=1)


In [36]:
emojis_data.head()

,#,emoji,unicode,name
0,1,😀,U+1F600,grinning face
1,2,😃,U+1F603,grinning face with big eyes
2,3,😄,U+1F604,grinning face with smiling eyes
3,4,😁,U+1F601,beaming face with smiling eyes
4,5,😆,U+1F606,grinning squinting face


In [45]:
# 入力した単語と最もnameとの類似度が高い絵文字を返す関数
def find_most_similar_emoji(input_word, emojis_data):
	input_vec = word_to_vec(input_word)
	similarities = []
	for i, row in emojis_data.iterrows():
		emoji_vec = word_to_vec(row["name"])
		similarity = cosine_similarity([input_vec], [emoji_vec])[0][0]
		similarities.append(similarity)
	max_index = np.argmax(similarities)
	print(f"max_index: {max_index}")
	return emojis_data.iloc[max_index]



In [46]:
# ユーザーからの入力
input_word = input("単語を入力してください: ")

# 最も近いデータを見つける
result = find_most_similar_emoji(input_word, emojis_data)

print("最も近い絵文字:", result["name"], result["emoji"])

max_index: 0
最も近い絵文字: grinning face 😀


In [17]:
# 入力単語に対して最も近い絵文字を出力する関数
def get_closest_emoji(word, n=4):
    word_vec = get_word_vector(word)
    emoji_vecs = [get_emoji_vector(emoji) for emoji in emoji_dict.values()]
    
    # 単語ベクトルと各絵文字ベクトルの類似度(コサイン類似度)を計算
    similarities = [F.cosine_similarity(word_vec, emoji_vec)[0] for emoji_vec in emoji_vecs]
    
    # 上位N件の類似度と対応する絵文字を出力
    top_indices = torch.topk(torch.tensor(similarities), n).indices
    for i in top_indices:
        emoji = list(emoji_dict.values())[i]
        name = list(emoji_dict.keys())[list(emoji_dict.values()).index(emoji)]
        print(f"{name}: {emoji} (similarity: {similarities[i]:.4f})")


In [22]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

# 絵文字の名前とUnicodeの対応辞書を作成
emoji_dict = {
    "grinning face": "\U0001f600",
    "smiling face with open mouth": "\U0001f603",
    "winking face": "\U0001f609",
    "robot face": "\U0001f916",
    # 他の絵文字も追加...
}

# emoji_dict に EmojiData からデータを追加
for i in range(len(EmojiData)):
    name = EmojiData["text"][i]
    emoji = EmojiData["image"][i]
    emoji_dict[name] = emoji

In [23]:
# emoji_dict の中身を出力
i = 0
for name, emoji in emoji_dict.items():
	print(f"{name}: {emoji}")
	i+=1
	if i == 10:
		break

grinning face: {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x00\x00\x00\x01\x00\x08\x02\x00\x00\x00\xd3\x10?1\x00\x00\xe2FIDATx\x9c\xdc\xfdy\xd8\xadYV\x17\x08\xfe\xd6\xda\xef{\xce\xf9\xc6;\xc6\x1c\x19\x199ENd\x92\x90\x90#I\x82\x82\x89  \x0e%Ti\xb5U*\x8f>\xd6ck[jY>\xb6]\xfdt?\xfah\x17mk\xd1*(\x8a"P\xcd`\x02\x82\x08E"\x90d\x92\xe4<\x06\xe4\x1c\xe3\x8d\x88\x1bq\xc7o:\xe7}\xf7Z\xfd\xc7Zk\xef}\xce\xf9\xbe{oDdbu\xbfy\xf3\x8bs\xde\xf3\x0e{\xaf\xbd\x86\xdf\x1a\xf6\xde\xa4\xaa\xb8\x85\xe3\x96.\x8a\x83\x00\x05\xe8\xd9\xdcr\xd3\xf7\xae7@\x8f;\xdf^O\xcd\xdfc\xaf9\xe9\xc9\'\xfdJk\xedY\x7fx{\xd0\xda\xd7[!\x0b\xc5\xf3\xd7\xaf|>\xf7>\xdbG\xdd\xf8\xe2\xf5\xae=\xb7\'\xffg?\xba\x1b\xff\xfc\xac\xf8\xfey\xdeu\x8b\xcf\xb9\x95\x87\xaf\xb0\xe9\x97\xb5=\xb8\xa1\x18\x14^|\xfelq+O8V\xfeop\xe5-\xb6\xeaY\xa9\xb3g\xf5\xe4\xff\xec\xc7M\x04\xe09\x1f\xcf\xbf\xff\xc7*o4Jn\xe5\xe2\xf5\xdbo\xfc\xe4\x9b^s\xe3{K\x07\x8f\xd5\x85\xede\xeb\xda\x91n\x81A\xf9\xe4\xb6\xdd\xe2qc\xe1?\x89\x02\xff\xbf\xc2\xb8_\xaa\xe

In [18]:

# BERTの読み込み
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 単語をベクトル化する関数
def get_word_vector(word):
    inputs = tokenizer(word, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# 絵文字をベクトル化する関数
def get_emoji_vector(emoji):
    inputs = tokenizer(emoji, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0]


# 使用例
get_closest_emoji("happy")

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [20]:
# 使用例
get_closest_emoji("robot")

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [10]:
from transformers import BertTokenizer, BertForMaskedLM
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_dataset

# データセットの読み込み
dataset = load_dataset("parquet", data_files={"train": "Data/EmojiData.parquet"})

Generating train split: 2749 examples [00:01, 1918.01 examples/s]


In [11]:
# トークナイザーとモデルの読み込み
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")



C:\Users\user\Jlab\EmojiSearch\EmojiSearch_venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
# データの前処理
def preprocess(examples):
    inputs = [f"word: {tokenizer.mask_token}" for word in examples["word"]]
    targets = examples["emoji"]
    model_inputs = tokenizer(inputs, return_tensors="pt", padding=True)
    labels = tokenizer.batch_encode_plus(targets, return_tensors="pt", padding=True)["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs



In [19]:
!pip install -U accelerate
!pip install -U transformers

  Using cached accelerate-0.30.0-py3-none-any.whl (302 kB)


In [24]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.40.2', '0.30.0')

In [23]:

args = TrainingArguments(output_dir="emoji_model", evaluation_strategy="epoch", per_device_train_batch_size=8)
trainer = Trainer(model=model, args=args, train_dataset=dataset["train"].map(preprocess))


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:

# 転移学習の実行
trainer.train()


In [ ]:

# 評価と推論
eval_results = trainer.evaluate(dataset["test"].map(preprocess))
print(f"Evaluation results: {eval_results}")

word = "happy"
inputs = tokenizer(f"word: {word}", return_tensors="pt")
output = model(**inputs)
logits = output.logits
predicted_emoji = tokenizer.decode(torch.argmax(logits, dim=-1)[0])
print(f"Predicted emoji for '{word}': {predicted_emoji}")